# MAUDE 데이터 클러스터링 분석

## ⚠️ GPU 필수 요구사항

**이 노트북은 NVIDIA GPU가 필수입니다.**

- RAPIDS (cuDF, cuML, cuPy) 라이브러리는 CUDA 지원 GPU에서만 작동합니다
- GPU 메모리: 최소 8GB 이상 권장  
- CUDA 버전: 11.x 또는 12.x 또는 13.x

GPU가 없는 환경에서는 CPU 기반 대안을 사용하세요:
- cuDF → Pandas
- cuML UMAP → umap-learn  
- cuML HDBSCAN → hdbscan (scikit-learn-contrib)

---

## 분석 개요

MAUDE 의료기기 부작용 데이터를 다음 파이프라인으로 클러스터링합니다:

1. **텍스트 임베딩**: SentenceTransformer (S-PubMedBert)
2. **차원 축소**: UMAP (768D → 15D)
3. **클러스터링**: HDBSCAN (밀도 기반)  
4. **결과 분석**: 클러스터별 특성 파악

---

## 필요 패키지 설치

```bash
# RAPIDS 라이브러리 (CUDA 버전에 맞게 선택)
# CUDA 11.x:
pip install cudf-cu11 cuml-cu11 cupy-cuda11x

# CUDA 12.x:
pip install cudf-cu12 cuml-cu12 cupy-cuda12x

# CUDA 13.x:
pip install cudf-cu13 cuml-cu13 cupy-cuda13x
```

## 1. 환경 설정

In [ ]:
# =====================
# 표준 라이브러리
# =====================
import sys
from pathlib import Path

# =====================
# 서드 파티 라이브러리
# =====================
import cudf
import cupy as cp
import cuml
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sentence_transformers import SentenceTransformer
from cuml import UMAP
from cuml.cluster import HDBSCAN
from cuml.metrics.cluster.silhouette_score import cython_silhouette_score
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from collections import Counter
import ast
import joblib

# =====================
# 경로 설정
# =====================
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"
OUTPUT_DIR = PROJECT_ROOT / "output"
FONT_DIR = PROJECT_ROOT / 'font'

# Python 내장 src 모듈과의 충돌 방지
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

print("✓ 라이브러리 임포트 완료")
print(f"✓ 프로젝트 루트: {PROJECT_ROOT}")

In [ ]:
import platform
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

# 로컬 폰트 지정
font_path = FONT_DIR / 'PretendardVariable.ttf'
fm.fontManager.addfont(str(font_path))
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()


## 2. 데이터 로드 및 전처리

### 2.1 데이터 로드

In [ ]:
# Gold 레이어 데이터 로드
df = pl.read_parquet(DATA_DIR / "silver" / "maude_text_processed.parquet")

print(f"데이터 크기: {df.shape}")
print(f"컬럼: {df.columns}")

### 2.2 클러스터링용 텍스트 생성 (mdr_sntc)

LLM이 추출한 4개 컬럼을 결합하여 임베딩용 텍스트를 생성합니다:
- `patient_harm`: 환자 피해 여부
- `problem_components`: 문제가 발생한 부품/컴포넌트
- `defect_confirmed`: 결함 확인 여부
- `defect_type`: 결함 유형

In [ ]:
df = (
    df
    .with_columns(
        # problem_components를 List에서 String으로 변환
        pl.when(pl.col("problem_components").is_not_null())
        .then(pl.col("problem_components").list.join(", "))
        .otherwise(pl.lit("unknown"))
        .alias("problem_components_str")
    )
    .with_columns(
        # mdr_sntc 생성: 4개 컬럼을 '. '로 결합
        pl.concat_str([
            pl.col("patient_harm").cast(pl.String).fill_null("unknown"),
            pl.col("problem_components_str"),
            pl.col("defect_confirmed").cast(pl.String).fill_null("unknown"),
            pl.col("defect_type").cast(pl.String).fill_null("unknown"),
        ], separator=". ").alias("mdr_sntc")
    )
    .drop("problem_components_str")
)

# 생성된 텍스트 예시
print("\n[mdr_sntc 예시]")
print(df.select("mdr_sntc").head(3))

## 3. 텍스트 임베딩 생성

### S-PubMedBert 모델
- 의료 도메인에 특화된 BERT 모델
- 768차원 벡터로 텍스트 임베딩
- 코사인 유사도 계산을 위해 정규화 수행

In [ ]:
%%time

# SentenceTransformer 모델 로드
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
print("✓ 모델 로드 완료")

# 텍스트 리스트 추출
texts = df.select("mdr_sntc")['mdr_sntc'].to_list()
print(f"✓ 텍스트 개수: {len(texts):,}")

# 임베딩 생성
embeddings = model.encode(
    texts,
    batch_size=32,              # GPU 메모리에 맞게 조정
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True   # L2 정규화 (코사인 유사도 최적화)
)

print(f"\n✓ 임베딩 생성 완료")
print(f"  - Shape: {embeddings.shape}")
print(f"  - Dtype: {embeddings.dtype}")

# 임베딩 저장
np.save(OUTPUT_DIR / "embeddings.npy", embeddings)
print(f"✓ 임베딩 저장: {OUTPUT_DIR / 'embeddings.npy'}")

# DataFrame에 임베딩 추가
df = df.with_columns(
    pl.Series("embeddings", embeddings.tolist())
)

## 4. UMAP 차원 축소

### UMAP (Uniform Manifold Approximation and Projection)
- 고차원 데이터의 구조를 보존하면서 저차원으로 축소
- 768차원 → 15차원으로 축소
- 클러스터링 성능과 계산 효율성 향상

### 파라미터 설명
- `n_neighbors=50`: 지역 구조를 파악할 이웃 개수 (클수록 전역 구조 중시)
- `min_dist=0.0`: 저차원 공간에서 점들의 최소 거리 (0에 가까울수록 밀집)
- `n_components=15`: 축소할 차원 수
- `metric='cosine'`: 거리 측정 방식 (텍스트 임베딩에 적합)

In [ ]:
%%time

# UMAP 모델 초기화
umap_model = UMAP(
    n_neighbors=50,
    min_dist=0.0,
    n_components=15,
    metric='cosine',
    random_state=42,
    verbose=True
)

# 차원 축소 수행 (GPU 가속)
X = umap_model.fit_transform(embeddings)
X = cp.asarray(X, dtype=cp.float32)

print(f"\n✓ UMAP 차원 축소 완료")
print(f"  - 입력 shape: {embeddings.shape}")
print(f"  - 출력 shape: {X.shape}")

# UMAP 모델 및 결과 저장
joblib.dump(umap_model, MODELS_DIR / "umap_model.joblib")
np.save(OUTPUT_DIR / "umap_X.npy", cp.asnumpy(X))

print(f"✓ UMAP 모델 저장: {MODELS_DIR / 'umap_model.joblib'}")
print(f"✓ 축소 결과 저장: {OUTPUT_DIR / 'umap_X.npy'}")

## 5. HDBSCAN 클러스터링

### 5.1 평가 함수 정의

클러스터링 품질을 측정하는 3가지 지표:
- **Silhouette Score**: 클러스터 응집도와 분리도 (-1 ~ 1, 높을수록 좋음)
- **Davies-Bouldin Index**: 클러스터 간 분리도 (0 ~ ∞, 낮을수록 좋음)
- **Calinski-Harabasz Index**: 분산 비율 (0 ~ ∞, 높을수록 좋음)

In [ ]:
def evaluate_clustering(X, labels, name):
    """
    클러스터링 결과 평가 함수
    
    Parameters
    ----------
    X : array-like
        특징 벡터
    labels : array-like
        클러스터 레이블 (-1은 noise)
    name : str
        클러스터링 방법 이름
    """
    # Noise 제외
    mask = labels != -1
    
    if len(set(labels[mask])) <= 1:
        print(f"{name}: 클러스터가 1개 이하입니다 (평가 불가)")
        return
    
    # 평가 지표 계산
    sil = cython_silhouette_score(X[mask], labels[mask])
    dbi = davies_bouldin_score(X[mask], labels[mask])
    chi = calinski_harabasz_score(X[mask], labels[mask])
    
    print(f"[{name}] Silhouette={sil:.3f}, DBI={dbi:.3f}, CHI={chi:.1f}")

### 5.2 HDBSCAN 하이퍼파라미터 그리드 서치

HDBSCAN의 핵심 파라미터:
- `min_cluster_size`: 클러스터로 간주할 최소 샘플 수 (클수록 큰 클러스터만 생성)
- `min_samples`: 핵심 포인트 판별 기준 (클수록 보수적, noise 증가)
- `metric`: 거리 측정 방식 (euclidean: UMAP 결과에 적합)

목표: **Silhouette Score가 가장 높은** 파라미터 조합 찾기

In [ ]:
def fit_until_k(X, target_k=18, 
                mcs_grid=(300, 500, 800, 1200, 2000, 3000, 5000, 8000, 12000),
                ms_grid=(1, 3, 5, 10)):
    """
    HDBSCAN 하이퍼파라미터 그리드 서치 (Silhouette Score 기준)
    
    Parameters
    ----------
    X : array-like
        특징 벡터 (UMAP 결과)
    target_k : int
        목표 클러스터 개수 상한 (이하의 결과만 고려)
    mcs_grid : tuple
        min_cluster_size 후보값들
    ms_grid : tuple
        min_samples 후보값들
    
    Returns
    -------
    dict
        최적 파라미터와 결과
    """
    best = None
    X_np = cp.asnumpy(X) if hasattr(X, 'get') else X

    for mcs in mcs_grid:
        for ms in ms_grid:
            # HDBSCAN 클러스터링 수행
            hdb = HDBSCAN(
                min_cluster_size=mcs,
                min_samples=ms,
                metric="euclidean",
                cluster_selection_method="eom",  # Excess of Mass
            )
            labels = hdb.fit_predict(X)
            labels_np = labels.get() if hasattr(labels, 'get') else labels

            # 클러스터 개수 및 noise 비율 계산
            k = len(np.unique(labels_np[labels_np != -1]))
            noise = float((labels_np == -1).mean())

            # 클러스터가 1개 이하이면 스킵
            if k < 2:
                print(f"mcs={mcs:>5}, ms={ms:>2} -> k={k:>3}, noise={noise:.3f}, silhouette=N/A")
                continue

            # Silhouette Score 계산 (noise 제외)
            mask = labels_np != -1
            sil = cython_silhouette_score(X_np[mask], labels_np[mask])

            print(f"mcs={mcs:>5}, ms={ms:>2} -> k={k:>3}, noise={noise:.3f}, silhouette={sil:.3f}")

            # 목표 클러스터 개수 이하이고, Silhouette Score가 가장 높은 것 선택
            if k <= target_k:
                if best is None or sil > best["silhouette"]:
                    best = {
                        "mcs": mcs, 
                        "ms": ms, 
                        "k": k, 
                        "noise": noise, 
                        "silhouette": sil, 
                        "labels": labels, 
                        "model": hdb
                    }

    return best

In [ ]:
%%time

# 그리드 서치 실행
print("HDBSCAN 하이퍼파라미터 그리드 서치 시작...\n")

grid_lst = (12000, 16000, 18000, 20000, 22000, 24000)
best = fit_until_k(X, target_k=10, ms_grid=(3, 5, 7, 9), mcs_grid=grid_lst)

# 최적 파라미터 출력
print("\n" + "="*60)
print("BEST 파라미터:")
print(f"  - min_cluster_size: {best['mcs']}")
print(f"  - min_samples: {best['ms']}")
print(f"  - 클러스터 개수: {best['k']}")
print(f"  - Noise 비율: {best['noise']:.1%}")
print(f"  - Silhouette Score: {best['silhouette']:.3f}")
print("="*60)

clusters = best["labels"]
hdbscan_model = best["model"]

### 5.3 모델 및 결과 저장

In [ ]:
# HDBSCAN 모델 저장
joblib.dump(hdbscan_model, MODELS_DIR / "hdbscan_model.joblib")
print(f"✓ HDBSCAN 모델 저장: {MODELS_DIR / 'hdbscan_model.joblib'}")

# 클러스터 레이블 저장
clusters_np = cp.asnumpy(clusters) if hasattr(clusters, 'get') else clusters
np.save(OUTPUT_DIR / "cluster_labels.npy", clusters_np)

print(f"✓ 클러스터 레이블 저장: {OUTPUT_DIR / 'cluster_labels.npy'}")
print(f"  - 총 샘플: {len(clusters_np):,}")
print(f"  - 고유 클러스터: {len(np.unique(clusters_np))} (noise 포함)")

### 5.4 클러스터링 평가

In [ ]:
# NumPy 배열로 변환
X_np = cp.asnumpy(X) if hasattr(X, "get") else X

# 평가 지표 출력
evaluate_clustering(X_np, clusters_np, "HDBSCAN")

# 클러스터별 샘플 수
unique, counts = np.unique(clusters_np, return_counts=True)
print("\n[클러스터별 샘플 수]")
for cid, count in sorted(zip(unique, counts), key=lambda x: -x[1]):
    label = "Noise" if cid == -1 else f"Cluster {cid}"
    print(f"  {label}: {count:>6,} ({count/len(clusters_np)*100:>5.2f}%)")

### 5.5 클러스터링 결과 시각화 (PCA 2D)

In [ ]:
# PCA로 2차원으로 축소 (시각화용)
Z = PCA(n_components=2).fit_transform(X_np)

# DataFrame 생성
df_temp = pd.DataFrame({
    "PC1": Z[:, 0],
    "PC2": Z[:, 1],
    "Cluster": clusters_np
})

# 시각화
plt.figure(figsize=(10, 8))
sns.scatterplot(
    data=df_temp, 
    x="PC1", 
    y="PC2", 
    hue="Cluster", 
    s=6, 
    legend="full",
    palette="tab10"
)
plt.title("HDBSCAN Clustering Result (PCA 2D Projection)", fontsize=14, fontweight='bold')
plt.xlabel("PC1", fontsize=12)
plt.ylabel("PC2", fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Cluster")
plt.tight_layout()
plt.show()

## 6. 클러스터링 결과 분석

### TODO: Noise 데이터 처리 방안 검토

현재 noise 비율이 ~38%로 높은 편입니다. 다음 방안들을 고려할 수 있습니다:

1. **Soft Clustering**: 각 noise 포인트를 가장 가까운 클러스터에 할당
2. **별도 분석**: Noise 그룹만 따로 2차 클러스터링 수행
3. **분류 모델 학습**: 클러스터링 결과로 분류기를 학습한 후 noise 예측
4. **임계값 조정**: `min_cluster_size`/`min_samples` 파라미터 재조정
5. **그대로 유지**: Noise는 이상치로 간주하고 분석에서 제외

---

### 6.1 클러스터 레이블 추가 및 저장

In [ ]:
# 원본 DataFrame에 클러스터 레이블 추가
df_cluster = df.with_columns(
    pl.Series("cluster", clusters_np)
)

# 클러스터별 개수 확인
cluster_counts = (
    df_cluster
    .group_by("cluster")
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

print("[클러스터별 샘플 수]")
print(cluster_counts)

In [ ]:
# 클러스터링 결과 저장
output_path = DATA_DIR / 'silver' / "maude_clustered.parquet"
df_cluster.write_parquet(output_path)

print(f"✓ 클러스터링 결과 저장: {output_path}")
print(f"  - 총 레코드: {df_cluster.shape[0]:,}")
print(f"  - 컬럼 수: {df_cluster.shape[1]}")

### 6.2 클러스터별 샘플 확인

In [ ]:
# 각 클러스터의 대표 샘플 확인
for cid in sorted(df_cluster["cluster"].unique()):
    if cid == -1:
        print(f"\n{'='*60}")
        print(f"Cluster: NOISE (-1)")
        print(f"{'='*60}")
    else:
        print(f"\n{'='*60}")
        print(f"Cluster {cid}")
        print(f"{'='*60}")
    
    samples = (
        df_cluster
        .filter(pl.col("cluster") == cid)
        .select("mdr_sntc")
        .head(3)
    )
    
    for i, row in enumerate(samples.iter_rows(), 1):
        print(f"[{i}] {row[0]}")

### 6.3 클러스터 대표 샘플 추출 (Centroid 기반)

**TODO: Representative 선택 방법 개선 검토**

현재는 Centroid(중심점)에 가장 가까운 샘플을 대표로 선택합니다.

개선 방안:
- **Medoid**: 클러스터 내 모든 포인트까지 거리 합이 최소인 실제 샘플
- **Diversity Sampling**: Centroid 근처 + 경계 샘플 등 다양한 특성을 포함

In [ ]:
def get_representatives_polars(df_cluster, X_np, labels_np, n=10):
    """
    클러스터별 대표 샘플 추출 (Centroid 기반)
    
    Parameters
    ----------
    df_cluster : pl.DataFrame
        클러스터 레이블이 포함된 DataFrame
    X_np : np.ndarray
        특징 벡터 (UMAP 결과)
    labels_np : np.ndarray
        클러스터 레이블
    n : int
        클러스터당 추출할 샘플 수
    
    Returns
    -------
    dict
        {cluster_id: DataFrame} 형태의 대표 샘플들
    """
    df_cluster_idx = df_cluster.with_row_index("row_id")
    reps = {}

    for cid in np.unique(labels_np):
        # Noise는 제외
        if cid == -1:
            continue

        # 해당 클러스터의 인덱스 및 특징 벡터
        idx = np.where(labels_np == cid)[0]
        
        # Centroid (중심점) 계산
        centroid = X_np[idx].mean(axis=0)
        
        # Centroid까지의 거리 계산
        dists = np.linalg.norm(X_np[idx] - centroid, axis=1)

        # 거리가 가까운 순서대로 n개 선택
        top_local = np.argsort(dists)[:n]
        top_idx = idx[top_local]

        # DataFrame 필터링
        reps[cid] = (
            df_cluster_idx
            .filter(pl.col("row_id").is_in(top_idx.tolist()))
            .drop("row_id")
        )

    return reps

# 클러스터별 대표 샘플 5개씩 추출
reps = get_representatives_polars(df_cluster, X_np, clusters_np, n=5)

print(f"✓ 클러스터별 대표 샘플 추출 완료 (각 {5}개)")
print(f"  - 추출된 클러스터 수: {len(reps)}")

In [ ]:
# 대표 샘플 확인용 컬럼 정의
MDR_COLS = [
    'product_code', 
    'event_type', 
    'patient_harm', 
    'problem_components', 
    'defect_confirmed', 
    'defect_type', 
    'mdr_sntc'
]

# 예시: 클러스터 0의 대표 샘플
if 0 in reps:
    print("[Cluster 0 대표 샘플]")
    print(reps[0][MDR_COLS])
else:
    print("클러스터 0이 존재하지 않습니다.")

## 7. 클러스터별 특성 분석

### 7.1 범주형 변수 분석 함수

In [ ]:
def analyze_categorical_by_cluster(df_cluster, categorical_col, cluster_col='cluster', top_n=None):
    """
    클러스터별 범주형 변수 비율 분석 및 시각화
    
    Parameters
    ----------
    df_cluster : pl.DataFrame
        클러스터 레이블이 포함된 DataFrame
    categorical_col : str
        분석할 범주형 변수 컬럼명
    cluster_col : str
        클러스터 컬럼명 (기본값: 'cluster')
    top_n : int, optional
        상위 N개 범주만 분석 (None이면 전체)
    
    Returns
    -------
    tuple
        (절대빈도 교차표, 비율 교차표)
    """
    # Top N 범주 필터링
    if top_n is not None:
        top_categories = (
            df_cluster
            .group_by(categorical_col)
            .agg(pl.len().alias('count'))
            .sort('count', descending=True)
            .head(top_n)
            .select(categorical_col)
            .to_series()
            .to_list()
        )

        df_filtered = df_cluster.filter(pl.col(categorical_col).is_in(top_categories))
        print(f"\n상위 {top_n}개 범주 선택: {top_categories}")
    else:
        df_filtered = df_cluster

    # 1. 교차표 생성 (절대 빈도)
    crosstab = (
        df_filtered
        .group_by([cluster_col, categorical_col])
        .agg(pl.len().alias('count'))
        .pivot(values='count', index=cluster_col, on=categorical_col)
        .fill_null(0)
    )

    print(f"\n=== {categorical_col} 절대 빈도 ===")
    print(crosstab)

    # 2. 클러스터별 비율 계산
    cluster_totals = (
        df_cluster
        .group_by(cluster_col)
        .agg(pl.len().alias('total'))
    )

    crosstab_with_total = crosstab.join(cluster_totals, on=cluster_col)
    cat_columns = [col for col in crosstab.columns if col != cluster_col]

    crosstab_pct = crosstab_with_total.with_columns([
        (pl.col(col) / pl.col('total') * 100).alias(f"{col}_pct")
        for col in cat_columns
    ]).select([cluster_col] + [f"{col}_pct" for col in cat_columns])

    print(f"\n=== {categorical_col} 클러스터별 비율 (%) ===")
    print(crosstab_pct)

    # 3. 시각화
    crosstab_pd = crosstab.to_pandas().set_index(cluster_col)

    plt.figure(figsize=(12, 6))
    crosstab_pd.plot(kind='bar', stacked=False)
    plt.title(f'클러스터별 {categorical_col} 분포', fontsize=14, fontweight='bold')
    plt.xlabel('Cluster', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title=categorical_col, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

    return crosstab, crosstab_pct

### 7.2 이벤트 유형별 분석

In [ ]:
analyze_categorical_by_cluster(df_cluster, 'event_type', 'cluster')

### 7.3 환자 피해 여부별 분석

In [ ]:
analyze_categorical_by_cluster(df_cluster, 'patient_harm')

### 7.4 제품 코드별 분석 (Top 10)

In [ ]:
analyze_categorical_by_cluster(df_cluster, 'product_code', top_n=10)

### 7.5 문제 컴포넌트 키워드 분석

In [ ]:
def cluster_keyword_unpack(df, col_name, cluster_col='cluster'):
    """
    클러스터별 키워드 빈도 분석 (리스트 타입 컬럼)
    
    Parameters
    ----------
    df : pl.DataFrame
        분석 대상 DataFrame
    col_name : str
        리스트가 들어있는 컬럼명 (예: 'problem_components')
    cluster_col : str
        클러스터 컬럼명 (기본값: 'cluster')
    
    Returns
    -------
    list
        클러스터별 키워드 빈도 Counter 객체 리스트
    """
    df_temp = df.select([cluster_col, col_name])

    # 문자열을 리스트로 변환 (필요한 경우)
    if df_temp[col_name].dtype == pl.Utf8:
        df_temp = df_temp.with_columns(
            pl.col(col_name)
            .map_elements(lambda x: ast.literal_eval(x) if x else [], return_dtype=pl.List(pl.Utf8))
        )

    # 리스트를 explode하여 키워드별로 분리
    exploded_df = (
        df_temp
        .explode(col_name)
        .filter(pl.col(col_name).is_not_null())
        .filter(pl.col(col_name) != "")
    )

    # 클러스터별 키워드 빈도 계산
    keyword_counts = (
        exploded_df
        .with_columns(
            pl.col(col_name).str.to_lowercase().str.strip_chars()
        )
        .group_by([cluster_col, col_name])
        .agg(pl.len().alias('count'))
        .sort([cluster_col, 'count'], descending=[False, True])
    )

    # 클러스터별로 Counter 생성
    unique_clusters = df[cluster_col].unique().sort()
    cluster_lst = []
    
    for cluster_id in unique_clusters:
        cluster_data = keyword_counts.filter(pl.col(cluster_col) == cluster_id)
        counts = Counter(dict(zip(
            cluster_data[col_name].to_list(),
            cluster_data['count'].to_list()
        )))
        cluster_lst.append(counts)

    # 결과 출력
    for i, counts in enumerate(cluster_lst):
        cid = unique_clusters[i]
        label = "NOISE" if cid == -1 else f"Cluster {cid}"
        
        print(f"\n{'='*60}")
        print(f"{label}")
        print(f"{'='*60}")
        print(f"총 키워드 수: {sum(counts.values()):,}")
        print(f"고유 키워드 수: {len(counts):,}")
        print(f"\nTop 10 키워드:")
        for keyword, count in counts.most_common(10):
            print(f"  {keyword:30s}: {count:>6,}")

    return cluster_lst

# 문제 컴포넌트 키워드 분석
keyword_result = cluster_keyword_unpack(df_cluster, 'problem_components', 'cluster')

## 8. 결과 요약

### 저장된 파일

1. **모델 파일** (`models/`)
   - `umap_model.joblib`: UMAP 차원 축소 모델
   - `hdbscan_model.joblib`: HDBSCAN 클러스터링 모델

2. **중간 결과** (`output/`)
   - `embeddings.npy`: SentenceTransformer 임베딩 (768D)
   - `umap_X.npy`: UMAP 축소 결과 (15D)
   - `cluster_labels.npy`: 클러스터 레이블

3. **최종 결과** (`data/silver/`)
   - `maude_clustered.parquet`: 클러스터 레이블이 추가된 전체 데이터

### 다음 단계

- [ ] Noise 데이터 처리 방안 결정
- [ ] Representative 샘플 선택 방법 개선
- [ ] 클러스터별 라벨링 (의미 부여)
- [ ] 통계 분석 및 인사이트 도출